In [5]:
from huggingface_hub import login
from dotenv import load_dotenv
import os
import torch
from transformers import pipeline
load_dotenv() 
login(token =os.getenv("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/kevin/.cache/huggingface/token
Login successful


In [6]:
pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b",
    device="mps",  # replace with "mps" to run on a Mac device
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 13.72it/s]


In [8]:
text = "Once upon a time,"
outputs = pipe(text, max_new_tokens=256)
response = outputs[0]["generated_text"]
print(response)

Once upon a time, there was a little girl who was born with a heart condition. She was born with a hole in her heart, and she had to have surgery to fix it. The surgery was successful, and the little girl was able to live a normal life.

The little girl was very happy, and she loved to play with her friends. She was also very smart, and she loved to read books. One day, the little girl’s parents took her to the doctor for a checkup. The doctor told them that the little girl’s heart was still not working properly. The doctor said that the little girl would need to have another surgery to fix her heart. The little girl was very sad, but she knew that she had to have the surgery. The little girl’s parents were very worried, but they knew that they had to do what was best for their daughter. The little girl had the surgery, and she was able to live a normal life.

The little girl was very happy, and she loved to play with her friends. She was also very smart, and she loved to read books. O